In [1]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import nltk
# nltk.download()
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [4]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout, MaxPool1D
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Bidirectional

In [5]:
# # Load your favorite word2vec model
# cont = Contractions('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
# text = "we're"
# text = list(cont.expand_texts([text], precise=True))[0]

In [6]:
def conv_dataframes(pos_path, neg_path):
    with open(pos_path,'r') as f:
        data_p = f.readlines()
    print(data_p[11])
    with open(neg_path,'r') as f:
        data_n = f.readlines()
    pos_data = shuffle(pd.DataFrame(data_p, columns = ["Doc Text"]))
#     pos_data['Sentiment'] = 1
#     pos_data.columns = ["Doc Text", "Sentiment"]
    neg_data = shuffle(pd.DataFrame(data_n, columns = ["Doc Text"]))
#     neg_data['Sentiment'] = -1
#     neg_data.columns = ["Doc Text", "Sentiment"]
    return pos_data, neg_data


## The code for pos tagging and lemmatize sentence is fron the following link:
 ### https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258

In [7]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [8]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#     print(wordnet_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
#         if tag is not None:
#             lemmatized_sentence.append(lemmatizer.lemmatize(word, tag)) 

    return " ".join(lemmatized_sentence)

In [9]:
# print(lemmatizer.lemmatize("I am loving it")) #I am loving it
# print(lemmatizer.lemmatize("loving")) #loving
# print(lemmatizer.lemmatize("loving", "v")) #love
# print(lemmatize_sentence("I am loving it")) #I be love it

In [10]:
def text_preprocessing(data):
    data["Doc Text"] = data["Doc Text"].str.replace('\n\n', ' ')
    data["Doc Text"] = data["Doc Text"].str.replace('[^\w\s]',' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\s+[a-zA-Z]\s+',' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\^[a-zA-Z]\s+',' ')

    data["Doc Text"] = data["Doc Text"].str.replace('\s+',' ')

    # Remove stop words from text
    data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_list]))
    
#     data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([lemmatizer.lemmatize(y) for y in x.split()]))
    data["Doc Text"] = data["Doc Text"].apply(lambda x: lemmatize_sentence(x))

    return data

In [33]:
def create_vocab(pos_data, neg_data):
    temp_pos = text_preprocessing(pos_data)
    temp_neg = text_preprocessing(neg_data)
#     temp_pos2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_pos['Doc Text']])
#     temp_neg2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_neg['Doc Text']]) 
#     temp_pos["Doc Text Tokens"] = temp_pos2 
#     temp_neg["Doc Text Tokens"] = temp_neg2
    return temp_pos, temp_neg

In [34]:
file_path  = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 594 Deep Learning for NLP\Assignments\rt-polaritydata' 
pos_path = os.path.join(file_path, 'rt-polarity.pos')
neg_path = os.path.join(file_path, 'rt-polarity.neg')

In [35]:
pos_data, neg_data = conv_dataframes(pos_path, neg_path)
pos_data.head()

what really surprises about wisegirls is its low-key quality and genuine tenderness . 



,Doc Text
1017,"human resources was a good , straightforward t..."
2176,an ingenious and often harrowing look at damag...
2686,"slow and ponderous , but rohmer's drama builds..."
1321,forgettable horror -- more gory than psycholog...
3057,"this is an exercise in chilling style , and tw..."


In [36]:
pos_data['Doc Text'][11]

'what really surprises about wisegirls is its low-key quality and genuine tenderness . \n'

In [37]:
# hm_lines = 5331

# tokenizer = nltk.RegexpTokenizer(r"\w+")
# spell = Speller(lang='en')
# # spell = SpellChecker()

# stop_words = set(stopwords.words('english'))
stop_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer() 

vocab_p, vocab_n = create_vocab(pos_data, neg_data)

vocab_p['Sentiment'] = 1
vocab_n['Sentiment'] = 0

In [38]:
vocab_p.head()

,Doc Text,Sentiment
1017,human resource good straightforward tale time ...,1
2176,ingenious often harrow look damaged people fam...,1
2686,slow ponderous rohmer drama build intense indo...,1
1321,forgettable horror gory psychological highly s...,1
3057,exercise chill style twohy film sub inside eye...,1


In [39]:
len(vocab_p)

5331

## Splitting the data into training, validation, and testing

In [40]:
vocab_p_train1, vocab_p_test= train_test_split(vocab_p, test_size = 0.2)
vocab_p_train, vocab_p_val = train_test_split(vocab_p_train1, test_size = 0.25)
vocab_p_test.head()

,Doc Text,Sentiment
3881,kind movie come along occasionally one unconve...,1
3103,way seep consciousness linger question film re...,1
2886,movie amateurish minor treat,1
3304,cautionary tale grandiosity college student se...,1
4646,stylish cast clever script solution help chica...,1


In [41]:
vocab_p_train.head()

,Doc Text,Sentiment
1114,surreal gilliam esque film also trouble interp...,1
1630,time existential drama without pretension asso...,1
3109,bottom line piece work brilliantly,1
2651,one thing read rail ongoing unprecedented cons...,1
4982,look intelligent movie release pent anger enou...,1


In [42]:
vocab_n_train1, vocab_n_test= train_test_split(vocab_n, test_size = 0.2)
vocab_n_train, vocab_n_val = train_test_split(vocab_n_train1, test_size = 0.25)
vocab_n_test.head()

,Doc Text,Sentiment
1312,pedestrian flat drama scream amateur almost ev...,0
251,road perdition love strangeness excellence,0
1765,exception fleetingly amusing improvisation ced...,0
4678,admirable rigor jimmy relentless anger script ...,0
3219,benigni pinocchio extremely straight mind numb...,0


In [43]:
training = shuffle(pd.concat([vocab_p_train,vocab_n_train])).reset_index(drop = True)
# training['Type'] = 'Training'
validation = pd.concat([vocab_p_val,vocab_n_val]).reset_index(drop = True)
# validation['Type'] = 'Validation'
test = pd.concat([vocab_p_test,vocab_n_test]).reset_index(drop = True)
# len(validation)

In [44]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Doc Text   2132 non-null   object
 1   Sentiment  2132 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.4+ KB


In [45]:
training.to_csv('Training Data.csv')
validation.to_csv('Validation.csv')
test.to_csv('Test Data.csv')

In [46]:
embeddings = {}
with open("glove.6B.300d.txt", 'r', encoding="utf-8") as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = asarray(values[1:], dtype='float32')
        embeddings[word] = vector

In [47]:
def emb_matrix(t,embeddings):
    # creating a embedding matrix for the words in training data, which will be used as weight matrix for embedding layer
    vocab_size = len(t.word_index) + 1    
    embedding_matrix = zeros((vocab_size, 300))
    for word, i in t.word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix, vocab_size

In [48]:
def model_vanilla_rnn(embedding_matrix, vocab_size, max_length, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(SimpleRNN(units = 300, activation = activation, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.add(Dropout(Dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    return model


In [49]:
def model_lstm(embedding_matrix, vocab_size, max_length, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(LSTM(units = 300, activation = activation, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.add(Dropout(Dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    return model


In [50]:
def model_gru(embedding_matrix, vocab_size, max_length, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(GRU(units = 300, activation = activation, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.add(Dropout(Dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    return model


In [ ]:
max_len = [4, 10, 20, 40, 60, 100]
batch_size= [32, 64, 128, 256]
learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
epochs = [5,10,15,20]
for i in max_len:
    for j in batch_size

random_search.fit(x_train, y_train)

In [29]:
{'batch_size': 0, 'epochs': 7, 'max_length': 59, 'padd': 0, 'trunc': 0}

{'batch_size': 0, 'epochs': 7, 'max_length': 59, 'padd': 0, 'trunc': 0}

## Taking elementwise max of the hidden states

In [32]:
def model_vanilla_rnn_em(embedding_matrix, vocab_size, max_length, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(SimpleRNN(units = 300, activation = activation,return_sequences=True, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.add(MaxPool1D(max_length, strides = None))
#     model.add(Dropout(Dropout_rate))

#     model.add(Dense(1, activation='softmax'))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    
#     print(z)
    return model


In [33]:
def model_lstm_em(embedding_matrix, vocab_size, max_length, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(LSTM(units = 300, activation = activation,return_sequences=True, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.add(MaxPool1D(max_length, strides = None))
#     model.add(Dropout(Dropout_rate))

#     model.add(Dense(1, activation='softmax'))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    
#     print(z)
    return model


In [34]:
def model_gru_em(embedding_matrix, vocab_size, max_length, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(GRU(units = 300, activation = activation,return_sequences=True, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.add(MaxPool1D(max_length, strides = None))
#     model.add(Dropout(Dropout_rate))

#     model.add(Dense(1, activation='softmax'))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    
#     print(z)
    return model


In [35]:
# max_length = 40
# epochs = 15
# batch_size = 128
# encoded_train = tokenise_tf.texts_to_sequences(training['Doc Text'])
# training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
# embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings)

# model = model_vanilla_rnn_em(embedding_matrix, vocab_size, max_length, 'tanh', 0.2, 0.001, 0.001)

# history = model.fit(training_padded, training['Sentiment'], epochs=epochs, verbose=1, batch_size=batch_size, shuffle =True, validation_split = 0.2)
# accuracy = history.history['val_accuracy'][-1]

In [55]:
def model_vanilla_rnn_bd(embedding_matrix, vocab_size, max_length, adam_optimizer, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(Bidirectional(SimpleRNN(units = 300, activation = activation, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef))))
    model.add(Dropout(Dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    return model


In [59]:
max_length = 40
epochs = 15
learning_rate = 0.001
batch_size = 64
encoded_train = tokenise_tf.texts_to_sequences(training['Doc Text'])
training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings)
adam_optimizer = optimizers.Adam(learning_rate=learning_rate)
model = model_vanilla_rnn_bd(embedding_matrix, vocab_size, max_length, adam_optimizer, 'tanh', 0.2, 0.001, 0.001)

history = model.fit(training_padded, training['Sentiment'], epochs=epochs, verbose=1, batch_size=batch_size, shuffle =True, validation_split = 0.25)
accuracy = history.history['val_accuracy'][-1]

Epoch 1/15
126/126 [==============================] - 42s 335ms/step - loss: 5.1197 - accuracy: 0.6622 - val_loss: 2.0090 - val_accuracy: 0.6917
Epoch 2/15
126/126 [==============================] - 41s 324ms/step - loss: 1.2526 - accuracy: 0.7154 - val_loss: 0.8993 - val_accuracy: 0.7223
Epoch 3/15
126/126 [==============================] - 42s 337ms/step - loss: 0.8545 - accuracy: 0.7227 - val_loss: 0.8080 - val_accuracy: 0.7123
Epoch 4/15
126/126 [==============================] - 42s 333ms/step - loss: 0.7917 - accuracy: 0.7279 - val_loss: 0.7485 - val_accuracy: 0.7322
Epoch 5/15
126/126 [==============================] - 41s 324ms/step - loss: 0.7515 - accuracy: 0.7346 - val_loss: 0.7596 - val_accuracy: 0.7093
Epoch 6/15
126/126 [==============================] - 41s 323ms/step - loss: 0.7265 - accuracy: 0.7335 - val_loss: 0.7388 - val_accuracy: 0.7207
Epoch 7/15
126/126 [==============================] - 42s 333ms/step - loss: 0.7380 - accuracy: 0.7310 - val_loss: 0.7363 - val_ac

In [65]:
def model_lstm_bd(embedding_matrix, vocab_size, max_length, adam_optimizer, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(Bidirectional(LSTM(units = 300, activation = activation, kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef))))
    model.add(Dropout(Dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    return model


In [69]:
max_length = 40
epochs = 20
learning_rate = 0.001
batch_size = 128
encoded_train = tokenise_tf.texts_to_sequences(training['Doc Text'])
training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings)
adam_optimizer = optimizers.Adam(learning_rate=learning_rate)
model = model_lstm_bd(embedding_matrix, vocab_size, max_length, adam_optimizer, 'tanh', 0.2, 0.001, 0.001)

history = model.fit(training_padded, training['Sentiment'], epochs=epochs, verbose=1, batch_size=batch_size, shuffle =True, validation_split = 0.25)
accuracy = history.history['val_accuracy'][-1]

Epoch 1/20
63/63 [==============================] - 74s 1s/step - loss: 9.0225 - accuracy: 0.6624 - val_loss: 1.3562 - val_accuracy: 0.7123
Epoch 2/20
63/63 [==============================] - 73s 1s/step - loss: 0.9291 - accuracy: 0.7110 - val_loss: 0.7748 - val_accuracy: 0.7314
Epoch 3/20
63/63 [==============================] - 76s 1s/step - loss: 0.7501 - accuracy: 0.7287 - val_loss: 0.7347 - val_accuracy: 0.7391
Epoch 4/20
63/63 [==============================] - 75s 1s/step - loss: 0.7241 - accuracy: 0.7325 - val_loss: 0.7345 - val_accuracy: 0.7207
Epoch 5/20
63/63 [==============================] - 74s 1s/step - loss: 0.7086 - accuracy: 0.7361 - val_loss: 0.7189 - val_accuracy: 0.7376
Epoch 6/20
63/63 [==============================] - 75s 1s/step - loss: 0.6910 - accuracy: 0.7414 - val_loss: 0.6886 - val_accuracy: 0.7406
Epoch 7/20
63/63 [==============================] - 75s 1s/step - loss: 0.6857 - accuracy: 0.7417 - val_loss: 0.6896 - val_accuracy: 0.7399
Epoch 8/20
63/63 [==

In [56]:
# def objective_func(args):
#     max_length = args['max_length']
# #     af_in_simpleRNN = args['af_in_simpleRNN']
#     batch_size = args['batch_size']
#     learning_rate = args['learning_rate']
# #     padd = args['padd']
# #     trunc = args['trunc']
#     epochs = args['epochs']
# #     kernel_regularizer_coef = args['kernel_regularizer_coef']
# #     activity_regularizer_coef = args['activity_regularizer_coef']
# #     Dropout_rate = args['Dropout_rate']
    
#     encoded_train = tokenise_tf.texts_to_sequences(training['Doc Text'])
#     training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
#     embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings)
#     adam_optimizer = optimizers.Adam(learning_rate=learning_rate)
#     model = model_vanilla_rnn_bd(embedding_matrix, vocab_size, max_length, adam_optimizer, 'tanh', 0.2, 0.001, 0.001)
    
#     history = model.fit(training_padded, training['Sentiment'], epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True, validation_split = 0.25)
#     accuracy = history.history['val_accuracy'][-1]
    
#     # summarize history for accuracy
#     plt.figure(figsize=(10,5))
#     plt.plot(history.history['accuracy'])
#     plt.plot(history.history['val_accuracy'])
#     plt.title('model accuracy')
#     plt.ylabel('accuracy')
#     plt.xlabel('epoch')
#     plt.legend(['training', 'validation'], loc='upper left')
#     plt.show()    
    
# #     encoded_validation = tokenise_tf.texts_to_sequences(validation['Doc Text'])

# #     validation_padded = pad_sequences(encoded_validation, maxlen=max_length, padding=padd, truncating = trunc)
# #     loss, accuracy = model.evaluate(validation_padded, validation['Sentiment'], verbose=0)
#     return -round((accuracy),2)

# space = {'max_length': hp.choice('max_length',range(4,100)), 
# #          'af_in_simpleRNN': hp.choice('af_in_simpleRNN', ['tanh', 'sigmoid']), 
#         'batch_size': hp.choice('batch_size', [64, 128, 256]),
# #          'padd': hp.choice('padd', ['pre', 'post']),
# #          'trunc': hp.choice('trunc', ['pre', 'post']),
#          'epochs': hp.choice('epochs',range(5,20)), 
#          'learning_rate': hp.uniform('learning_rate', 0,1)
# #          'Dropout_rate': hp.uniform('Dropout_rate', 0, 1),
# #          'kernel_regularizer_coef': hp.uniform('kernel_regularizer_coef', 0, 10),
# #          'activity_regularizer_coef': hp.uniform('activity_regularizer_coef', 0, 10)
#         }
                                
                                
# tokenise_tf = Tokenizer()
# tokenise_tf.fit_on_texts(training['Doc Text'])                                
# best_vanilla_rnn_bd = fmin(objective_func, space, algo=tpe.suggest, max_evals=10)
# print(best_vanilla_rnn_bd)

  0%|          | 0/10 [20:12<?, ?trial/s, best loss=?]


KeyboardInterrupt: 